In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
import multiprocessing as mp
import argparse
import os

In [10]:
decade_lists=list(range(1800,2010,10))

In [43]:
fcat_files=glob.glob(f'/data/dharp/compounds/datasets/*_fcat.txt')

df_list=[]
for cur_file in fcat_files:
    cur_df=pd.read_table(cur_file,header=None,usecols=[0])
    df_list.append(cur_df)
    
fcat_df=pd.concat(df_list)
fcat_df.columns=['fname']
fcat_df[['fnum']]=fcat_df.fname.str.split('/',expand=True)[[6]]
fcat_df.fnum=fcat_df.fnum.str.rstrip('.pkl')
actual_list=fcat_df.fnum.unique()

In [45]:
len(actual_list)

10347

In [11]:
pkl_files=glob.glob(f'/datanaco/dharp/compounds/datasets/googleV3/*/*pkl')
len(pkl_files)

pkl_files_dict = dict()
for cur_file in pkl_files: 
    fsize = os.stat(cur_file).st_size
    pkl_files_dict[cur_file] = fsize

In [46]:
size_df=pd.DataFrame.from_dict(pkl_files_dict,orient='index')
size_df.reset_index(inplace=True)
size_df.columns=['fname','fsize']
size_df[['year','fnum']]=size_df.fname.str.split('/',expand=True)[[6,7]]
size_df.fnum=size_df.fnum.str.rstrip('.pkl')
stored_list=size_df.fnum.unique()
len(stored_list)

10347

In [23]:
size_df.groupby(['year'])['fsize'].sum()/10**9

year
1800     19.417731
1810     25.523244
1820     33.505400
1830     48.393158
1840     54.132005
1850     66.263352
1860     70.823276
1870     83.395534
1880     96.430940
1890    110.505459
1900    127.379251
1910    134.541142
1920    134.267649
1930    133.477596
1940    137.622296
1950    159.168369
1960    189.301311
1970    200.787911
1980    201.181671
1990    206.857616
2000    220.492505
2010    208.638266
Name: fsize, dtype: float64

In [4]:
to_divide_list=size_df.sort_values(ascending=False,by='fsize',ignore_index=True)
to_divide_list['row_perc']=to_divide_list.fsize/to_divide_list.fsize.sum()*100
to_divide_list

,fname,fsize,row_perc
0,/datanaco/dharp/compounds/datasets/googleV3/20...,176840358,8.020243e-02
1,/datanaco/dharp/compounds/datasets/googleV3/20...,143092593,6.489681e-02
2,/datanaco/dharp/compounds/datasets/googleV3/20...,137636894,6.242248e-02
3,/datanaco/dharp/compounds/datasets/googleV3/20...,123689770,5.609704e-02
4,/datanaco/dharp/compounds/datasets/googleV3/20...,121483805,5.509657e-02
...,...,...,...
10333,/datanaco/dharp/compounds/datasets/googleV3/20...,993,4.503554e-07
10334,/datanaco/dharp/compounds/datasets/googleV3/20...,992,4.499019e-07
10335,/datanaco/dharp/compounds/datasets/googleV3/20...,991,4.494484e-07
10336,/datanaco/dharp/compounds/datasets/googleV3/20...,990,4.489949e-07


In [6]:
to_divide_list.iloc[0].fname

'/datanaco/dharp/compounds/datasets/googleV3/2000/602.pkl'

In [6]:
maxvalue = 1

lastvalue = 0
newcum = []
labels=[]
cur_label=1
for row in to_divide_list.itertuples():
    thisvalue =  row.row_perc + lastvalue
    if thisvalue > maxvalue:
        thisvalue = 0
        cur_label+=1
    newcum.append(thisvalue)
    labels.append(cur_label)
    lastvalue = thisvalue
to_divide_list['fcat']=labels
to_divide_list

,fname,fsize,row_perc,fcat
0,/datanaco/dharp/compounds/datasets/googleV3/60...,4824711805,1.043950e-01,1
1,/datanaco/dharp/compounds/datasets/googleV3/10...,3460349063,7.487353e-02,1
2,/datanaco/dharp/compounds/datasets/googleV3/10...,3459980866,7.486556e-02,1
3,/datanaco/dharp/compounds/datasets/googleV3/59...,3277271564,7.091218e-02,1
4,/datanaco/dharp/compounds/datasets/googleV3/62...,3062609103,6.626740e-02,1
...,...,...,...,...
10342,/datanaco/dharp/compounds/datasets/googleV3/12...,2118,4.582836e-08,100
10343,/datanaco/dharp/compounds/datasets/googleV3/55...,2008,4.344823e-08,100
10344,/datanaco/dharp/compounds/datasets/googleV3/61...,2002,4.331841e-08,100
10345,/datanaco/dharp/compounds/datasets/googleV3/97...,1454,3.146102e-08,100


In [20]:
to_divide_list.iloc[10].fname

'/datanaco/dharp/compounds/datasets/googleV3/601.pkl'

In [35]:
to_divide_list.loc[to_divide_list.fcat.between(1,25)]

,fname,fsize,row_perc,fcat
0,/datanaco/dharp/compounds/datasets/googleV3/80...,7677431026,0.155029,1
1,/datanaco/dharp/compounds/datasets/googleV3/79...,7228628958,0.145966,1
2,/datanaco/dharp/compounds/datasets/googleV3/80...,7134667418,0.144069,1
3,/datanaco/dharp/compounds/datasets/googleV3/14...,7076838209,0.142901,1
4,/datanaco/dharp/compounds/datasets/googleV3/81...,7050167074,0.142362,1
...,...,...,...,...
197,/datanaco/dharp/compounds/datasets/googleV3/29...,5882970430,0.118793,25
198,/datanaco/dharp/compounds/datasets/googleV3/18...,5881815323,0.118770,25
199,/datanaco/dharp/compounds/datasets/googleV3/81...,5881468020,0.118763,25
200,/datanaco/dharp/compounds/datasets/googleV3/18...,5862185384,0.118374,25


In [1]:
to_divide_list.loc[to_divide_list.fcat.between(1,10)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/latinum_fcat.txt',header=False,index=False,sep="\t")




NameError: name 'to_divide_list' is not defined

In [14]:
to_divide_list.loc[to_divide_list.fcat.between(11,30)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/drogium_fcat.txt',header=True,index=False,sep="\t")




In [15]:
to_divide_list.loc[to_divide_list.fcat.between(31,50)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/mithril_fcat.txt',header=True,index=False,sep="\t")




In [16]:
to_divide_list.loc[to_divide_list.fcat.between(51,70)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/adamant_fcat.txt',header=True,index=False,sep="\t")



In [17]:
to_divide_list.loc[to_divide_list.fcat.between(71,90)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/rivium_fcat.txt',header=True,index=False,sep="\t")



In [18]:
to_divide_list.loc[to_divide_list.fcat.between(91,100)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/tiberium_fcat.txt',header=True,index=False,sep="\t")
